In [40]:
import json
from io import StringIO
import os
import pandas as pd
import pyarrow
from src.mtbl_globals import DIR_EXTRACT, DIR_TRANSFORM

In [41]:
positions = {}
for file in os.listdir(DIR_TRANSFORM):
    file_path = os.path.join(DIR_TRANSFORM, file)
    pos = file.split('_')[1].split('.')[0]
        
    df = pd.read_json(file_path, orient='table', dtype_backend='pyarrow')
    positions[pos] = df

NameError: name 'ArrowDtype' is not defined